In [15]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

Analyte History, Plot Looking Observation Outliers
--

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

pd.set_option("display.max_columns",25)

In [17]:
siteID = 'WI201'
years = '10'
days = str(int(years)*365)
minobs = 5 #minimum number of observations to calculate what is typical
types = ['Al-w', 'As-w', 'B-w', 'Br-w', 'Ca-w', 'Cs-w', 'Cl-w', 'F-w', 'Fe-w', 'HCO3-w', 'H2S-w', 'K-w', 'Li-w', 'Mg-w', 'Na-w', 'NH3-w', 'NO3-N-w', 'PO4-P-w', 'Rb-w', 'SO4-w', 'SiO2-w']

**Get all data**

In [18]:
#setup dataframes
names = ['date', 'obs']
dfall = pd.DataFrame() #for background
dflast = pd.DataFrame() #for last values

#loop through typeIDs
for typeID in types:
    url = 'https://fits.geonet.org.nz/observation?siteID='+siteID+'&typeID='+typeID+'&days='+days
    df = pd.read_csv(url, skiprows=1, parse_dates=True, names=names, usecols=[0,1], index_col=0)
    df['typeID'] = typeID
    dfall = dfall.append(df)

In [19]:
dates = dfall.index.unique()

In [20]:
dtab = dfall.pivot(columns='typeID', values='obs')

In [21]:
#replace zero values with NaN
dtab.replace(to_replace=0, value=np.nan, inplace=True)  

In [22]:
#loop through dataframe, passing on extra row to 'data' each time, analyse 'data'
for rows in range(minobs+1,len(dtab)+1):
#for rows in range(9,12):
    data = dtab.iloc[0:rows]
    
    #drop columns with null final value
    droplist = [col for col in data.columns if pd.isnull(data[col].iloc[-1])]
    data.drop(droplist, axis='columns', inplace=True)
        
    #drop columns with insufficient non-null values
    droplist = [col for col in data.columns if pd.notnull(data[col]).sum() < minobs+1]
    data.drop(droplist, axis='columns', inplace=True)
    
    finaldate = data.index[-1].strftime('%Y-%m-%d')
    print 'rows='+str(rows), 'date= '+finaldate
    #get last values
    dlast = data.iloc[-1:]
    #unpivot last values and set up for plotting
    dlastp = pd.melt(dlast)
    dlastp.index += 1
    #drop last value from from data
    data.drop(data.index[-1:], inplace=True)
                
    #plot
    flierprops = dict(marker='o', markerfacecolor='black', markersize=2)
    whiskerprops = dict(linestyle='-')         
    bp = dlastp.plot(figsize=(30,10), marker='o', color='green', linestyle='none', legend=False)
    data.boxplot(ax=bp, whis=[5,95], fontsize=14, flierprops=flierprops, whiskerprops=whiskerprops)
    bp.get_figure().suptitle("") #get rid of automatic boxplot sub-title
    bp.set_ylabel("Concentration (mg/L)", fontsize=14)
    bp.set_xlabel("typeID", fontsize=14)
    bp.set_yscale('log')
    bp.set_title(siteID+' for '+finaldate, fontsize=20)
    fig = bp.get_figure()
    fig.savefig('/home/sherburn/Desktop/'+siteID+'_'+finaldate+'.png', dpi=200)
    plt.close(fig)

rows=6 date= 2008-05-02
rows=7 date= 2008-08-06
rows=8 date= 2008-10-21
rows=9 date= 2008-12-11
rows=10 date= 2009-02-03
rows=11 date= 2009-03-10
rows=12 date= 2009-05-14
rows=13 date= 2009-08-11
rows=14 date= 2009-11-13
rows=15 date= 2010-03-02
rows=16 date= 2010-05-02
rows=17 date= 2010-08-11
rows=18 date= 2010-11-24
rows=19 date= 2011-01-12
rows=20 date= 2011-05-20
rows=21 date= 2011-08-18
rows=22 date= 2011-09-23
rows=23 date= 2014-09-17
rows=24 date= 2014-10-24
rows=25 date= 2014-11-20
rows=26 date= 2015-02-25
rows=27 date= 2015-06-05
rows=28 date= 2015-09-17
rows=29 date= 2015-12-23
rows=30 date= 2017-07-26
rows=31 date= 2017-08-18
